# RF-DETR: Roboflow Detection Transformer Tutorial

This tutorial demonstrates how to use the RF-DETR model for object detection tasks. RF-DETR is an enhanced version of the DETR (Detection Transformer) model with improvements focused on faster convergence and easier fine-tuning.

## Setup

Let's start by installing the library (if you haven't already):

In [ ]:
# Install the library (uncomment if needed)
# !pip install git+https://github.com/HasanGoni/objdetect-rf-detr.git

In [ ]:
# Import required libraries
import torch
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import requests
from io import BytesIO

# Import from our package
from objdetect.models import create_model
from objdetect.utils import plot_detections

## Load a Pre-trained RF-DETR Model

Let's create an RF-DETR model pre-trained on the COCO dataset:

In [ ]:
# Create the model
model = create_model("rfdetr", num_classes=91, pretrained=True)

# Set to evaluation mode
model.eval()

# Check if CUDA is available and move model to GPU if possible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

print(f"Model created and running on {device}")

## Load COCO Class Names

Let's load the COCO class names for better visualization:

In [ ]:
# COCO class names
COCO_CLASSES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

## Load and Preprocess a Test Image

In [ ]:
# Function to load an image from URL
def load_image_from_url(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img

# Load a test image
image_url = "https://raw.githubusercontent.com/ultralytics/yolov5/master/data/images/zidane.jpg"
image = load_image_from_url(image_url)

# Display the original image
plt.figure(figsize=(10, 6))
plt.imshow(image)
plt.axis('off')
plt.title('Original Image')
plt.show()

## Run Inference with RF-DETR

In [ ]:
# Set threshold for detection confidence
threshold = 0.7

# Run inference
with torch.no_grad():
    predictions = model.predict(image)
    
# Extract the predictions for the first (and only) image
prediction = predictions[0]

# Print detection summary
scores = prediction['scores']
labels = prediction['labels']
boxes = prediction['boxes']

# Filter by threshold
mask = scores >= threshold
filtered_scores = scores[mask]
filtered_labels = labels[mask]
filtered_boxes = boxes[mask]

print(f"Detected {len(filtered_scores)} objects with confidence >= {threshold}:")
for i in range(len(filtered_scores)):
    class_name = COCO_CLASSES[filtered_labels[i]]
    print(f"  {class_name}: {filtered_scores[i]:.2f} at {filtered_boxes[i].tolist()}")

## Visualize the Detections

In [ ]:
# Visualize using our utility function
fig, ax = plot_detections(image, prediction, class_names=COCO_CLASSES, score_threshold=threshold)
plt.title('RF-DETR Object Detection Results')
plt.show()

## Fine-tuning on a Custom Dataset

The following example shows how you would fine-tune RF-DETR on a custom dataset. This is a placeholder for demonstration - you would need your own dataset to actually run this code.

In [ ]:
# Create a model with custom number of classes (for example, 5 classes)
custom_model = create_model("rfdetr", num_classes=5, pretrained=True)

# Define fine-tuning parameters
dataset_path = "path/to/your/dataset"  # Replace with actual path
output_dir = "./fine_tuned_model"
epochs = 10
batch_size = 8
lr = 0.0001

# This would actually run if you had a prepared dataset
# metrics = custom_model.fine_tune(
#     dataset_path=dataset_path,
#     output_dir=output_dir,
#     epochs=epochs,
#     batch_size=batch_size,
#     lr=lr
# )

## Conclusion

In this tutorial, we've demonstrated how to:

1. Load a pre-trained RF-DETR model
2. Run inference on test images
3. Visualize the detection results
4. Outline the fine-tuning process for custom datasets

RF-DETR provides a clean API for object detection tasks with the power of transformer-based architecture, making it easier to detect objects accurately in various scenarios.